# BVA Figures for FRETBursts Paper

*This notebook generates the BVA figures for the paper **FRETBursts: Open Source Burst Analysis Toolkit for Confocal Single-Molecule FRET**.*

# Loading the software

We start loading the **`FRETBursts`** software:

In [ ]:
from fretbursts import *
sns = init_notebook()

In [ ]:
full_fname1= '001_20dA_20dT_200mM_MgCl.sm'
full_fname2= '002_HP3_RT_400mM_NaCl.sm'

In [ ]:
import os
if os.path.isfile(full_fname1):
    print ("Perfect, I found the file!")
else:
    print ("Sorry, I can't find the file:\n", full_fname1)

if os.path.isfile(full_fname2):
    print ("Perfect, I found the file!")
else:
    print ("Sorry, I can't find the file:\n", full_fname2)

# Load the selected file

In [ ]:
d1 = loader.usalex(full_fname1)
d2 = loader.usalex(full_fname2)

# us-ALEX parameters

At this point, in `d`, we only have the timestamps (`ph_times_t`) and the detector numbers (`det_t`):

In [ ]:
d1.add(det_donor_accept=(0, 1), 
      alex_period=4000, 
      D_ON=(2100, 3900), 
      A_ON=(150, 1900),
      offset=700)
bpl.plot_alternation_hist (d1)
loader.usalex_apply_period(d1)

In [ ]:
d2.add(det_donor_accept=(0, 1), 
      alex_period=4000, 
      D_ON=(2100, 3900), 
      A_ON=(150, 1900),
      offset=700)
bpl.plot_alternation_hist (d2)
loader.usalex_apply_period(d2)

The previous alternation histogram looks correct, so we can apply the current parameters:

In [ ]:
d1.calc_bg(bg.exp_fit, time_s=50.1, tail_min_us='auto', F_bg=1.7)
d2.calc_bg(bg.exp_fit, time_s=50.1, tail_min_us='auto', F_bg=1.7)

# Burst Variance Analysis

In [ ]:
d1.burst_search(m=10, computefret=False, ph_sel=Ph_sel(Dex='DAem'))
d1.calc_fret(count_ph=True, corrections=False)

In [ ]:
d2.burst_search(m=10, computefret=False, ph_sel=Ph_sel(Dex='DAem'))
d2.calc_fret(count_ph=True, corrections=False)

In [ ]:
ds1 = d1.select_bursts(select_bursts.naa, th1=30, computefret=False)
ds_1 = ds1.select_bursts(select_bursts.size, th1=30, computefret=False)
ds_FRET1 = ds_1.select_bursts(select_bursts.S, S1=0.25, S2=0.85, computefret=False)

In [ ]:
ds2 = d2.select_bursts(select_bursts.naa, th1=70, computefret=False)
ds_2 = ds2.select_bursts(select_bursts.size, th1=70, computefret=False)
ds_FRET2 = ds_2.select_bursts(select_bursts.S, S1=0.25, S2=0.85, computefret=False)

In [ ]:
dx=ds_FRET1
alex_jointplot(dx)
plt.savefig('ALEX_static.png', dpi=200, bbox_inches='tight')

In [ ]:
dx=ds_FRET2
alex_jointplot(dx)
plt.savefig('ALEX_dynamic.png', dpi=200, bbox_inches='tight')

In [ ]:
bursts1 =ds_FRET1.mburst[0]
ph_d1 = ds_FRET1.get_ph_times(ph_sel=Ph_sel(Dex='DAem'))
AemDex_mask1 = ds_FRET1.get_ph_mask(ph_sel=Ph_sel(Dex='Aem')) 
Dex_mask1 = ds_FRET1.get_ph_mask(ph_sel=Ph_sel(Dex='DAem'))   
AemDex_mask_d1 = AemDex_mask1[Dex_mask1]

In [ ]:
bursts_d1 = bursts1.recompute_index_reduce(ph_d1)

In [ ]:
n=7         # Determine the number of photons inside each sub-burst

Std_small1=[]
Avg_small1=[]
for burst1 in bursts_d1:
    E_small1=[]
    startlist = range(burst1.istart, burst1.istop + 2 - n, n)
    stoplist = [i + n for i in startlist]
    for start, stop in zip(startlist, stoplist): 
        A_D = AemDex_mask_d1[start:stop].sum()
        assert stop-start == n
        E = A_D / n
        E_small1.append(E)
    Std_small1.append(np.std(E_small1))

In [ ]:
sns.set_style(style='darkgrid')
plt.figure(figsize=(6,6))
x=np.arange(0,1.01,0.01)
y=np.sqrt((x*(1-x))/n)
plt.plot(x,y, lw=3, color='red')
im = sns.kdeplot(ds_FRET1.E_, np.array(Std_small1), shade=True, cmap='viridis', shade_lowest=False)
plt.xlim(0,1)
plt.ylim(0,0.4)
plt.xlabel('PR', fontsize=14)
plt.ylabel(r'$\sigma_E$', fontsize=24)
plt.savefig('BVA_static.png', dpi=200, bbox_inches='tight')

In [ ]:
bursts2 =ds_FRET2.mburst[0]
ph_d2 = ds_FRET2.get_ph_times(ph_sel=Ph_sel(Dex='DAem'))
AemDex_mask2 = ds_FRET2.get_ph_mask(ph_sel=Ph_sel(Dex='Aem')) 
Dex_mask2 = ds_FRET2.get_ph_mask(ph_sel=Ph_sel(Dex='DAem'))   
AemDex_mask_d2 = AemDex_mask2[Dex_mask2]

In [ ]:
bursts_d2 = bursts2.recompute_index_reduce(ph_d2)

In [ ]:
n=7         # Determine the number of photons inside each sub-burst

Std_small2=[]
Avg_small2=[]
for burst2 in bursts_d2:
    E_small2=[]
    startlist = range(burst2.istart, burst2.istop + 2 - n, n)
    stoplist = [i + n for i in startlist]
    for start, stop in zip(startlist, stoplist): 
        A_D = AemDex_mask_d2[start:stop].sum()
        assert stop-start == n
        E = A_D / n
        E_small2.append(E)
    Std_small2.append(np.std(E_small2))

In [ ]:
sns.set_style(style='darkgrid')
plt.figure(figsize=(6,6))
x=np.arange(0,1.01,0.01)
y=np.sqrt((x*(1-x))/n)
plt.plot(x,y, lw=3, color='red')
im = sns.kdeplot(ds_FRET2.E_, np.array(Std_small2), shade=True, cmap='viridis', shade_lowest=False)
plt.xlim(0,1)
plt.ylim(0,0.4)
plt.xlabel('PR', fontsize=14)
plt.ylabel(r'$\sigma_E$', fontsize=24)
plt.savefig('BVA_dynamic.png', dpi=200, bbox_inches='tight')

In [ ]:
import sys
from PIL import Image

In [ ]:
for label in ('static','dynamic'):
    ALEX= Image.open("ALEX_%s.png" % label)
    BVA = Image.open("BVA_%s.png" % label)
    ALEX.thumbnail((1200,1088))
    BVA.thumbnail((1160,1088))
    new_im = Image.new('RGBA', (2560,1088))
    im_list=[ALEX, BVA]
    j=0
    for im in im_list:
        new_im.paste(im, (j,0))
        j=j+1400

#    new_im.show()
    new_im.save('figures/ALEX_BVA_%s/ALEX_BVA_%s.png' % (label,label), bbox_inches='tight')